In [1]:
import json
from collections import Counter
from tqdm import tqdm
import pickle
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 100
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import sys
import os
import argparse
from os.path import dirname, realpath

import hashlib
import datetime
import datetime as datetime
import sklearn
import statistics
sys.path.append("./src/")
#from disrisknet.utils.parsing import parse_args
import pdb

src_path = "G:\\FillmoreCancerData\\markhe\\VTERisk" 
src_path2 = "G:\\FillmoreCancerData\\markhe\\VTERisk - Copy" 

def md5(key):
    return hashlib.md5(repr(key).encode()).hexdigest()

from sklearn.metrics import confusion_matrix

In [2]:
testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/allChars_DF.csv'))
from sklearn.metrics import confusion_matrix


In [3]:
logpath = os.path.join(src_path ,'logs_transformer_vte/F/Ran10_15/Index', "929e0ce070ea788e9df2c5d0f393347d.results.test_preds")


In [4]:
 
with open(logpath, 'rb') as f:
    R6 = pickle.load(f)
    p6 = np.array(R6['probs'])

pat_ids = (testDF['patient_id'] ).astype(int)
 

In [5]:
R6.keys()

dict_keys(['golds', 'probs', 'patient_golds', 'exams', 'pids', 'dates', 'censor_times', 'days_to_final_censors'])

In [29]:
p = p6[:,1]

In [30]:
len(p)

24693

In [34]:
testDF['pids'] = pat_ids.apply(md5)

g6 = pd.DataFrame({'p6': np.array(R6['probs'])[:,1] ,    
                   'pids':np.array(R6['pids']) ,    
                   'model_date':np.array(R6['dates'])
                  })

In [35]:
M = pd.merge(g6, testDF)

m = M[["patient_id","dx_date", "index_date", "model_date",  "outcome","outcome_date","p6" ]]
m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])

m_VE = m[ m['outcome'] ]
m_VE1 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) < datetime.timedelta(730)]
m_VE0 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) >= datetime.timedelta(730)]
m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)  
m0= m[ ~ m['outcome'] ]

m1 = pd.concat([m_VE1, m_VE0, m0])

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_34916\3624253825.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_34916\3624253825.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)


In [36]:
R6.keys()

dict_keys(['golds', 'probs', 'patient_golds', 'exams', 'pids', 'dates', 'censor_times', 'days_to_final_censors'])

In [37]:
M.keys()

Index(['p6', 'pids', 'model_date', 'Unnamed: 0', 'patient_id', 'outcome',
       'dob', 'outcome_date', 'obs_time_end', 'index_date', 'dx_date',
       'obs_time_start', 'ev_length', 'len0', 't0', 'len30', 't30', 'len90',
       't90', 'len180', 't180', 'ks', 'sex', 'bmi', 'race'],
      dtype='object')

In [38]:
M

,p6,pids,model_date,Unnamed: 0,patient_id,outcome,dob,outcome_date,obs_time_end,index_date,...,len30,t30,len90,t90,len180,t180,ks,sex,bmi,race
0,0.059421,9c4a171786576cc81aef7a300c0d53cf,2009-03-03T00:00:00,49432,1009107581,False,1920-07-13,2009-12-06,2009-12-06,2009-03-03,...,938,2813 days,938,2813 days,938,2813 days,3,0,4,1
1,0.096332,bce1cd81e5f596e28b9d86c6306947d0,2010-05-26T00:00:00,91351,1006064340,False,1930-02-13,2017-08-05,2017-08-05,2010-05-26,...,553,4015 days,553,4015 days,553,4015 days,4,0,1,5
2,0.102915,81c191396aa5757ec3c1dce5914511e6,2009-08-25T00:00:00,97039,1007261854,False,1926-03-19,2015-03-30,2015-03-30,2009-08-25,...,975,3818 days,975,3818 days,975,3818 days,3,0,1,5
3,0.079554,b36fbdc43dba9a7e7f984c47315042de,2019-12-17T00:00:00,45797,1010730671,False,1939-12-29,2021-02-20,2021-02-20,2019-12-17,...,1121,4737 days,1121,4737 days,1121,4737 days,0,0,4,5
4,0.119963,26bd272f9e8bc1b7c13856e4cd4ae574,2012-09-11T00:00:00,7690,1014882035,True,1947-03-24,2012-12-28,2013-10-31,2012-09-11,...,623,1985 days,454,1925 days,231,1844 days,5,0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24688,0.068163,f52af8c831e0fa925154740a87265bb6,2015-03-31T00:00:00,90647,1004880538,False,1956-09-17,2015-10-26,2015-10-26,2015-04-01,...,904,1175 days,904,1175 days,904,1175 days,0,0,1,1
24689,0.196113,8bec5d8d74a575c91d9d0821816afd81,2006-06-09T00:00:00,3837,1009821184,False,1955-02-09,2006-06-19,2006-06-19,2006-06-09,...,369,1369 days,369,1369 days,369,1369 days,4,0,1,1
24690,0.264993,fa672ed2ee9cc54ac3736c713d5d9db3,2008-02-08T00:00:00,38560,1012036946,False,1927-05-20,2008-03-16,2008-03-16,2008-02-06,...,846,1062 days,846,1062 days,846,1062 days,4,0,4,1
24691,0.280431,050b7af9d4ff7056ea9e85a0c3e84ae7,2016-12-14T00:00:00,22056,1001945567,False,1942-01-24,2017-06-15,2017-06-15,2016-12-14,...,2418,6448 days,2418,6448 days,2418,6448 days,4,0,1,5


In [39]:
sklearn.metrics.roc_auc_score(m1['outcome'],  m1['p6'], average = 'samples')


0.6211496507485663

In [43]:
( confusion_matrix (m1['outcome'],  m1['p6']>.1) )

array([[ 6755, 15926],
       [  295,  1717]], dtype=int64)

In [76]:
# how many people had event within 6 months of index date

In [77]:
m1.to_csv('output/CC8_test180.csv') 

In [8]:
#m1.to_excel('output/dxMD_Sensitivity90.xlsx') 

In [27]:
statistics.quantiles( M['len180'],n=20)

[289.0,
 390.0,
 471.0,
 541.0,
 610.0,
 677.0,
 743.0,
 809.0,
 879.0,
 952.0,
 1036.0,
 1128.0,
 1230.0,
 1343.0,
 1482.5,
 1647.0,
 1880.9,
 2189.0,
 2772.0]

In [23]:
plt.hist(M['len180'])

(array([2.2715e+04, 1.8050e+03, 1.3000e+02, 2.5000e+01, 1.2000e+01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 2.0000e+00]),
 array([1.00000e+00, 2.37810e+03, 4.75520e+03, 7.13230e+03, 9.50940e+03,
        1.18865e+04, 1.42636e+04, 1.66407e+04, 1.90178e+04, 2.13949e+04,
        2.37720e+04]),
 <BarContainer object of 10 artists>)